# Tool for manipulating tensors using the hierarchical fiber abstraction

First, include some libraries

In [ ]:
# Run boilerplate code to set up environment

%run prelude.py

## Display a tensor


Following is an example of reading in a tensor and displaying it

In [ ]:
        
# Display an example tensor

filename = datafileName("draw-a.yaml")

f = open(filename)
for line in f:
    print(line.rstrip('\n'))
f.close()

In [ ]:
a = Tensor(filename)

displayTensor(a)

## Traverse a tensor

In [ ]:
# Traverse a tensor

a = Tensor(datafileName("matrix-a.yaml"))

displayTensor(a)

In [ ]:
a_m = a.getRoot()


for m, (a_k) in a_m:
    print(f"({m}, {a_k})")
    for k, (a_val) in a_k:
        print(f"Processing: ({k}, {a_val})")

# Element-wise update (empty) tensor, i.e., copy

In [ ]:
# Element-wise update a tensor

a = Tensor(datafileName("elementwise-a.yaml"))
z = Tensor(rank_ids=["M"])

displayTensor(a)
displayTensor(z)


In [ ]:
a_m = a.getRoot()
z_m = z.getRoot()

print("Z < A Fiber")

for m, (z_ref, a_val) in z_m << a_m:
    print(f"Processing: ({m}, ({z_ref}, {a_val})")
    
    z_ref += a_val

    displayTensor(z)

# Intersection

In [ ]:
# Fiber instersection

a = Tensor(datafileName("elementwise-a.yaml"))
b = Tensor(datafileName("elementwise-b.yaml"))

displayTensor(a)
displayTensor(b)

In [ ]:
a_m = a.getRoot()
b_m = b.getRoot()

print("Fiber a_m & b_m")

z_m = a_m & b_m

displayTensor(z_m)

## Element-wise multiply

In [ ]:
# Element-wise multiply

a = Tensor(datafileName("elementwise-a.yaml"))
b = Tensor(datafileName("elementwise-b.yaml"))
z = Tensor(rank_ids=["M"])

displayTensor(a)
displayTensor(b)

In [ ]:
a_m = a.getRoot()
b_m = b.getRoot()
z_m = z.getRoot()

print("Z < A Fiber")

for m, (z_ref, (a_val, b_val)) in z_m << (a_m & b_m):
    print(f"Processing: ({m}, ({z_ref}, ({a_val}, {b_val})))")

    z_ref += a_val * b_val

displayTensor(z)

## Dot-product

Here is a dot product of two tensors

In [ ]:
# Dot product
#
# To perform a dot-product we need a "row" for an output.
# So we represent the vectors as 2-D tensors
#


a = Tensor(datafileName("dot-product-a.yaml"))
b = Tensor(datafileName("dot-product-b.yaml"))
z = Tensor(rank_ids=["M"])

display(TensorImage(a).im)
display(TensorImage(b).im)

In [ ]:
a_m = a.getRoot()
b_m = b.getRoot()
z_m = z.getRoot()


for m, (z_ref, (a_k, b_k)) in z_m << (a_m & b_m):
    for k, (a_val, b_val) in a_k & b_k:
        print(f"Processing: [{k} -> ( {z_ref}, ({a_val}, {b_val})]")

        z_ref += a_val * b_val


display(TensorImage(z).im)

# Union

In [ ]:
# Fiber union

a = Tensor(datafileName("elementwise-a.yaml"))
b = Tensor(datafileName("elementwise-b.yaml"))

displayTensor(a)
displayTensor(b)

In [ ]:
a_m = a.getRoot()
b_m = b.getRoot()

print("Fiber a_m | b_m")

z_m = a_m | b_m

displayTensor(z_m)

## Element-wise addition

Following...



In [ ]:
#
# Do a sum of sums of the rows of two matrices
#

a = Tensor(datafileName("dot-product-a.yaml"))
b = Tensor(datafileName("dot-product-b.yaml"))

z = Tensor(rank_ids=["M"])

displayTensor(a)
displayTensor(b)

In [ ]:
a_m = a.getRoot()
b_m = b.getRoot()
z_m = z.getRoot()


for m, (z_ref, (mask_k, a_k, b_k)) in z_m << (a_m | b_m):
    for k, (ab_mask, a_val, b_val) in a_k | b_k:
        print(f"Processing: [{k} -> ( {z_ref}, ({ab_mask}, {a_val}, {b_val})]")

        z_ref += a_val + b_val


displayTensor(z)

## Reduce vector to a rank zero tensor

In [ ]:
a = Tensor(datafileName("elementwise-a.yaml"))
z = Tensor(rank_ids=[])

print("Tensor a")
displayTensor(a)

print("Tensor z")
displayTensor(z)

a_m = a.getRoot()
z_ref = z.getRoot()

for m_coord, (a_val) in a_m:
    z_ref += a_val

print("Result")
displayTensor(z)

## Testing area

For running alternative algorithms